In [ ]:
import simplenote as sn
from io import StringIO
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import ast
from IPython.core.display import display, HTML
import itertools
%matplotlib inline

pd.options.mode.chained_assignment = None
reference_date=pd.datetime.strptime("2017-12-10","%Y-%m-%d")
#reference_date=dt.datetime.now()-dt.timedelta(days=4) # always cehck everything from yesterday's reference (which has already passed)
simplenote = sn.Simplenote('mohdjamal8453@gmail.com', 'simple123')
taskNoteKey="41d06e8ced6c42389127e0d727974230"
ilmNoteKey="5f7cb643ec884642b250954ff3996f8c"
ilmRevLogKey="9ad07e0dc9e84909b17795e101683c68"
monthlyGoalNoteKey="ba7047bd84204ab49e9e271a5b164675"
habitKey="7b7f2636562c4f28b42f942ab2b6210d"
habitLogKey="d9a739e149bc4169a90379436a7cfe1c"
habitRevisionActionMap='6bf90144f253442bbcf102779268973b'

In [285]:
def get_current_habits():
    """
    Reads habit metadata from simmple note and returns the same
    :return: habit metadata (data frame)
    """
    habitraw = simplenote.get_note(habitKey)
    habit = habitraw[0]['content'].splitlines()[1:]
    habit = [ln.replace("*", "").strip() for ln in habit]
    habitcsv = "\n".join(habit)
    dathabit = pd.read_csv(StringIO(habitcsv))
    dathabit.columns = [x.strip() for x in dathabit.columns]
    dathabit.name = dathabit.name.apply(lambda x: x.strip().lower())
    dathabit.streak_type = dathabit.streak_type.apply(lambda x: x.strip().lower())
    dathabit.small_desc = dathabit.small_desc.apply(lambda x: x.strip())
    dathabit.status = dathabit.status.apply(lambda x: x.strip().lower())
    # dathabit.review_cycle=dathabit.review_cycle.apply(lambda x:x.strip().lower())
    # dathabit.missed_review_cycle=dathabit.missed_review_cycle.apply(lambda x:x.strip().lower())
    dathabit.start_date = dathabit.start_date.apply(lambda x: x.strip().lower())
    dathabit.start_date = pd.to_datetime(dathabit.start_date, errors="coerce", format="%d/%m/%y")
    return dathabit

get_current_habits()

,id,name,small_desc,streak_type,status,start_date,review_cycle,missed_review_cycle
0,1,shukr,always shukr na kalema dil ma ane zabaan ma,count,now,2017-12-13,30,7
1,2,ft,focused work with end time,day,focus,2017-12-13,30,0
2,3,goal,work towards your goal (with himmat),day,focus,2017-12-13,30,0
3,4,ilm,revise (n read) as per the plan,day,now,2017-12-13,30,0
4,5,quran,hifz towards goal ane murajaat as per schedule,day,now,2017-12-13,30,0
5,6,morning,simple morning routine,day,later,2017-12-13,30,0
6,7,gtd,complete task in timely manner,day,focus,2017-12-13,30,0
7,8,gussa,gussa (n ducha) ne gili jao ane action ma takh...,count,now,2017-12-13,30,7
8,9,budget,monthly budget n expense control expense (cheh...,count,now,2017-12-13,30,14
9,10,nazarulmaqam,habit of putting nazarul makam for every impor...,count,now,2017-12-13,30,7


In [154]:
def get_habit_log(reference_date):
    """
    Reads habit log from simmple note and returns the same

    Rules to interpret a habit log record:
    * no star, no double dash and non empty string (after stripping) - it is date - so update current date
    * no comma - means its pass with no comment (/reason) - so simply add ",,p"
    * 1 comma - means it is pass with comments (stored in reason) - simply add ",p"
    * 2 comma - nothing to do (it is fail)
    * if no star and double hash - update current year --2017--

    :return: habit log (as data frame)
    """

    habitLograw = simplenote.get_note(habitLogKey)
    habitLog = habitLograw[0]['content'].splitlines()[1:]
    habit_log_str = ""
    current_year = dt.datetime.now().year
    current_date = ""
    for ln in habitLog:
        ln = ln.strip()
        if ln == "": continue
        if '*' in ln:
            num_of_commas = ln.count(",")
            if num_of_commas == 0:
                ln += ",,p"
            elif num_of_commas == 1:
                ln += ",p"
        elif '--' in ln:
            current_year = int(ln.replace("--", ""))
        else:
            current_date = ln + ' ' + str(current_year)
        habit_log_str = habit_log_str + "\n" + current_date + "," + ln
    habit_log_str = habit_log_str.splitlines()
    habit_log_str = [ln.replace("*", "") for ln in habit_log_str if ln.count(",") == 3]
    dathabitLog = pd.read_csv(StringIO('\n'.join(habit_log_str)), header=None,
                              names=['date', 'name', 'reason', 'status'])
    dathabitLog.columns = [ln.strip() for ln in dathabitLog.columns]
    dathabitLog['name'] = dathabitLog.name.apply(lambda x: str(x).lower().strip())
    dathabitLog['reason'] = dathabitLog.reason.apply(lambda x: str(x).lower().strip())
    dathabitLog['status'] = dathabitLog.status.apply(lambda x: str(x).lower().strip())
    dathabitLog.date = pd.to_datetime(dathabitLog.date, format='%b %d %Y')
    dathabitLog = dathabitLog.loc[dathabitLog.date <= reference_date, :]  # filter logs greater than reference_date
    dathabitLog = dathabitLog.sort_values(axis=0, by=['name', 'date'], ascending=False)
    is_earlier_then_ref_date = dathabitLog.date.apply(lambda x, reference_date: x.date() <= reference_date.date(),
                                                      args=[reference_date])
    dathabitLog = dathabitLog.loc[is_earlier_then_ref_date, :]
    return dathabitLog

get_habit_log(reference_date)

,date,name,reason,status
3,2017-12-10,shukr,nan,p
6,2017-12-09,shukr,nan,p
8,2017-12-08,shukr,did a great job,p
9,2017-12-08,shukr,not practical,f
10,2017-12-08,shukr,not practical,f
11,2017-12-07,shukr,nan,p
12,2017-12-07,shukr,nan,p
15,2017-12-06,shukr,other urgent work,p
17,2017-12-05,shukr,nan,p
0,2017-12-10,goal,nan,p


In [153]:
def get_metrics_for_a_habit_streak(log_of_a_habit, streak_type, reference_date):
    if(streak_type== "day"):
        tmp_gpb=log_of_a_habit.groupby(['date'], as_index=False).mean().sort_values('date', ascending=False)
        tmp_gpb=tmp_gpb.reset_index(drop=True)
        tmp_gpb.loc[tmp_gpb.status<1,'status']=0
        tmp_gpb['days_from_now']=tmp_gpb.apply(lambda x: (reference_date-x.date).days if x.status==1 else -1,axis=1)
        days_delta=tmp_gpb.days_from_now[tmp_gpb.days_from_now>0].reset_index(drop=True)
        days_delta_shifted=days_delta.shift(periods=-1)
        streak=days_delta_shifted-days_delta
        streak_gpb=[sum(1 for _ in l) for n, l in itertools.groupby(streak)]
        max_streak=max(streak_gpb)+1
        #last_break_length=0
        if days_delta[0]>0:
            last_break_length=days_delta[0]
        else:
            index_last_fail=streak.index[streak>1][0]
            last_break_length=int(days_delta[index_last_fail+1]-days_delta[index_last_fail]-1)

        cur_streak= 0 if days_delta[0]!=0 else ((streak_gpb[0]+1) if streak[0]==1 else 1)
    else: # assume it is count
        days_delta = pd.Series([int(x) for x in log_of_a_habit.status])
        cur_streak = days_delta.index[days_delta == 0][0]
        streak_gpb = [sum(1 for _ in l) for n, l in itertools.groupby(days_delta)]
        max_streak = max(streak_gpb)
        last_break_length = streak_gpb[1] if days_delta[0] == 1 else streak_gpb[0]
    return {'cur_streak':cur_streak,'max_streak':max_streak,'last_break_length':last_break_length}

def get_streak_data_for_all_habits(habit_metadata, habit_log, reference_date):
    # name, current_streak, max_streak, last_break_length, last_reason, priority, days_from_start, status
    habit_streak = pd.DataFrame(
        columns=['small_desc', 'current_streak', 'max_streak', 'last_break_length', 'last_reason', 'days_from_start',
                 'habit_status'], index=habit_metadata.name)
    # print(habit_streak)
    for index, row in habit_metadata.iterrows():
        # print(row)
        habit_name = row['name']
        log_of_a_habit = habit_log[habit_log.name == habit_name].reset_index(drop=True)
        log_of_a_habit['status'] = (log_of_a_habit.status == 'p')
        habit_streak.loc[habit_name, 'small_desc'] = row['small_desc']
        habit_streak.loc[habit_name, 'habit_status'] = row['status']
        habit_streak.loc[habit_name, 'streak_type'] = row['streak_type']
        habit_streak.loc[habit_name, 'days_from_start'] = (reference_date - row['start_date']).days
        if log_of_a_habit.shape[0] > 0:
            streak_metric = get_metrics_for_a_habit_streak(log_of_a_habit, row['streak_type'], reference_date)
            habit_streak.loc[habit_name, 'current_streak'] = streak_metric['cur_streak']
            habit_streak.loc[habit_name, 'max_streak'] = streak_metric['max_streak']
            habit_streak.loc[habit_name, 'last_break_length'] = streak_metric['last_break_length']

    habit_streak.current_streak.fillna(0, inplace=True)
    habit_streak.max_streak.fillna(0, inplace=True)
    habit_streak.last_break_length.fillna(0, inplace=True)
    habit_streak.last_reason.fillna("", inplace=True)
    habit_streak = habit_streak.reset_index()
    return habit_streak

get_streak_data_for_all_habits(dathabit,dathabitLog, reference_date)

,name,small_desc,current_streak,max_streak,last_break_length,last_reason,days_from_start,habit_status,streak_type
0,shukr,always shukr na kalema dil ma ane zabaan ma,3,4,2,,-3,now,count
1,ft,focused work with end time,0,2,1,,-3,focus,day
2,goal,work towards your goal (with himmat),0,4,1,,-3,focus,day
3,ilm,revise (n read) as per the plan,0,0,0,,-3,now,day
4,quran,hifz towards goal ane murajaat as per schedule,0,0,0,,-3,now,day
5,morning,simple morning routine,0,0,0,,-3,later,day
6,gtd,complete task in timely manner,0,0,0,,-3,focus,day
7,gussa,gussa (n ducha) ne gili jao ane action ma takh...,0,0,0,,-3,now,count
8,budget,monthly budget n expense control expense (cheh...,0,0,0,,-3,now,count
9,nazarulmaqam,habit of putting nazarul makam for every impor...,0,0,0,,-3,now,count


In [166]:
def display_html_table(df, table_config):
    # given a dataframe
    # need to print it using html table, with certain column or a column has some font (color coding, strikethrough etc to share extra info)
    # display table

    #df = habit_action
    df = df.sort_values(by=table_config['column_sort_by'], ascending=table_config['column_sort_by_is_asc'])
    df.reset_index(drop=True, inplace=True)

    font_start_tag = "<font face='verdana' size='2' {text_font_attribute}><p align='{text_alignment}'>"
    font_end_tag = "</p></font>"
    # html_horizontal_ruler_in_table="<tr><td colspan='100'><hr/></td></tr>"
    html_horizontal_ruler_in_table = "<tr style='border-bottom:1px solid black'><td colspan='100%'></td></tr>"

    font_attributes = df.apply(table_config['func_giving_font_attribute'], axis=1)
    if table_config['column_name_whose_value_change_adds_horizontal_ruler'] is not None:
        val_change_index_for_horizontal_ruler = [sum(1 for _ in l) for n, l in itertools.groupby(
            df[table_config['column_name_whose_value_change_adds_horizontal_ruler']])]
        val_change_index_for_horizontal_ruler = [x - 1 for x in
                                                 pd.Series(val_change_index_for_horizontal_ruler).cumsum()][:-1]
    else:
        val_change_index_for_horizontal_ruler = []

    status = "<h3 align='center' style='color:grey;'>{table_heading}</h3>".format(
        table_heading=table_config['table_heading'])
    status += "<table width='100%'>"
    status += " ".join(
        ["<col width='{col_width}'>".format(col_width=str(x) + "%") for x in table_config['widths_in_pcntg']])
    status += "<tr>"
    status += " ".join(
        ["<th><p align='left'>{col_name}</p></th>".format(col_name=x) for x in table_config['column_headers']])
    status += "</tr>"

    for i in range(df.shape[0]):
        status += "<tr>"
        for j in range(len(table_config['column_name_in_order_of_display'])):
            column_name = table_config['column_name_in_order_of_display'][j]
            text_font_attribute = font_attributes[i] if table_config[
                                                            'column_name_needing_custom_formatting'] == column_name else ""
            text_alignment = table_config['text_alignment'][j]
            status += (
            "<td>" + font_start_tag.format(text_font_attribute=text_font_attribute, text_alignment=text_alignment) +
            str(df.loc[i, column_name]) + font_end_tag + "</td>")
        status += "</tr>"
        if i in val_change_index_for_horizontal_ruler:
            status += html_horizontal_ruler_in_table
    status += "</table>"
    display(HTML(status))

In [175]:

# Ordered by:
# priority, current/days (its so long and yet habit not formed)

# Color coded:
# Red - focus and current_streak <= 5
# Orange - now or later and current_streak <= 5
# black - current_streak > 5 and <= 30 (for all)
# green - current_streak > 30 (for all)
# horizontal ruler after focus
def display_habit_status(habit_status):
    

    def get_font_attribute(df):
        color = ""
        if df.current_streak <= 5:
            color = 'red' if df.habit_status == 'focus' else 'orange'
        elif df.current_streak <= 30:
            color = 'black'
        else:
            color = 'green'
        return 'color=' + color

    table_config = ({'column_name_in_order_of_display': ['name', 'small_desc', 'streak_str', 'habit_status'],
                     'column_headers': ['Name', 'Description', 'Streak', 'Focus'],
                     'column_sort_by': ['habit_status_as_priority_number', 'current_streak_by_total_days'],
                     'column_sort_by_is_asc': [True, True],
                     'widths_in_pcntg': [10, 60, 20, 10],
                     'text_alignment': ['left', 'left', 'left', 'left'],
                     'column_name_whose_value_change_adds_horizontal_ruler': 'habit_status',
                     'column_name_needing_custom_formatting': 'small_desc',
                     'func_giving_font_attribute': get_font_attribute,
                     'table_heading': 'Current Status'
                     })
    display_html_table(habit_status, table_config)

display_habit_status(habit_status)

In [169]:
def get_current_habit_status():
    # Status (simple status for awareness could be used for revision as well)
    # name, streak_str, description, priority, current_streak_by_total_days, current_streak
    # Goal, 4 current (3 best, 22 days)
    # Ordered by priority, current/days (its so long and yet habit not formed)
    # Color coded - red critical - just started or just failed - current streak < 5
    # red - current streak < 7 and priority - 1
    # orange - current streak < 7 and priority -2,3
    # black - current streak > 7 and < 30
    # green - streak > 30
    # horizontal ruler after focus

    habit_status=habit_streak.loc[:,['name','small_desc','habit_status','current_streak']]
    habit_status['streak_str']=habit_streak.apply(lambda x:"{current_streak}c ({max_streak}m, {days_from_start}d)".format(current_streak=x.current_streak,max_streak=x.max_streak,days_from_start=x.days_from_start),axis=1)
    habit_status['current_streak_by_total_days']=habit_streak.apply(lambda x:x.current_streak/x.days_from_start,axis=1)
    habit_status['habit_status_as_priority_number']=habit_status.habit_status.apply(lambda x:1 if x=='focus' else (2 if x=='now' else 3))
    #habit_status=habit_status.sort_values(by=['habit_status','current_streak_by_total_days']).reset_index(drop=True)
    return habit_status

habit_status=get_current_habit_status()
habit_status

,name,small_desc,habit_status,current_streak,streak_str,current_streak_by_total_days,habit_status_as_priority_number
0,ft,focused work with end time,focus,0,"0c (2m, -3d)",-0.0,1
1,goal,work towards your goal (with himmat),focus,0,"0c (4m, -3d)",-0.0,1
2,gtd,complete task in timely manner,focus,0,"0c (0m, -3d)",-0.0,1
3,habit,act towards making life long habit,focus,0,"0c (0m, -3d)",-0.0,1
4,morning,simple morning routine,later,0,"0c (0m, -3d)",-0.0,3
5,shukr,always shukr na kalema dil ma ane zabaan ma,now,3,"3c (4m, -3d)",-1.0,2
6,ilm,revise (n read) as per the plan,now,0,"0c (0m, -3d)",-0.0,2
7,quran,hifz towards goal ane murajaat as per schedule,now,0,"0c (0m, -3d)",-0.0,2
8,gussa,gussa (n ducha) ne gili jao ane action ma takh...,now,0,"0c (0m, -3d)",-0.0,2
9,budget,monthly budget n expense control expense (cheh...,now,0,"0c (0m, -3d)",-0.0,2


In [162]:
def get_actions_against_habits(dathabitLog,habit_streak,reference_date):
    # Action
    # name, weekday, reason, action
    # - pass and comment (add revise action for each if date within 2 days from now)
    # - fail (add action corresponding to reason if date within 3 days from now) - FT only for 1st day
    # - color code actions:
    # Ordered by: status and entry date
    # Color code: orange - failed yesterday (so most important to revise)
    # black - older dates
    # green - positive ones

    # prepare revise action map
    habit_rev_action_map_raw=simplenote.get_note(habitRevisionActionMap)
    habit_rev_action=habit_rev_action_map_raw[0]['content'].splitlines()[1:]
    habit_rev_action=[ln.replace("*","").strip() for ln in habit_rev_action]
    habit_rev_action_csv="\n".join(habit_rev_action)
    habit_rev_action=pd.read_csv(StringIO(habit_rev_action_csv))
    habit_rev_action.columns=[x.strip().lower() for x in habit_rev_action.columns]
    habit_rev_action.reason=habit_rev_action.reason.apply(lambda x:str(x.strip().lower()))
    habit_rev_action.action=habit_rev_action.action.apply(lambda x:x.strip())
    habit_rev_action.action_subsequent=habit_rev_action.action_subsequent.apply(lambda x:x.strip())

    #Since only last 3 days data affects, so let's filter only last 3 days logs
    # now for all those day based habits where we don't have expilicit entry let's make explicit no entry with forget reason
    last3_days_habit_log=dathabitLog[dathabitLog.date.apply(lambda x:x.date()>=(reference_date-dt.timedelta(days=2)).date())]

    #cartesian product - for getting date X name
    date_last_3_days = [reference_date.date() - dt.timedelta(days=x) for x in range(0, 3)]
    active_habit_names=habit_streak.name[(habit_streak.habit_status!="later") & (habit_streak.streak_type=="day")]
    df1 = pd.DataFrame({'key':[1 for x in range(len(date_last_3_days))], 'date':date_last_3_days})
    df2 = pd.DataFrame({'key':[1 for x in range(len(active_habit_names))], 'name':active_habit_names})
    ideal_habit_log=pd.merge(df1, df2,on='key')[['date', 'name']]
    ideal_habit_log.date=pd.to_datetime(ideal_habit_log.date)
    ideal_habit_log=pd.merge(ideal_habit_log,last3_days_habit_log,on=['date','name'],how='left')
    ideal_habit_log=pd.merge(ideal_habit_log,habit_streak[['name','current_streak','max_streak','days_from_start','habit_status','small_desc']],on=['name'],how='left')
    ideal_habit_log.reason=ideal_habit_log.apply(lambda x: x.reason if not pd.isnull(x.status) else ('forgot' if x.max_streak>0 else 'not started'),axis=1)
    ideal_habit_log.status.fillna('f',inplace=True)
    ideal_habit_log.loc[ideal_habit_log.reason=='nan','reason']=''
    #ideal_habit_log

    action_date_considered=pd.to_datetime(date_last_3_days)
    action_date_considered=action_date_considered.sort_values(ascending=False)
    # get action from last 2 days log
    habit_action_pass=ideal_habit_log.loc[(ideal_habit_log.date.isin(action_date_considered[0:2])) & (ideal_habit_log.status=='p') & (ideal_habit_log.reason!=''),['date','name','small_desc','habit_status','reason','status']]
    habit_action_fail=ideal_habit_log.loc[(ideal_habit_log.date.isin(action_date_considered[0:3])) & (ideal_habit_log.status=='f'),['date','name','small_desc','habit_status','reason','status']]
    habit_action=pd.concat([habit_action_pass,habit_action_fail],axis=0)
    habit_action['days_from_today']=abs(habit_action.date.apply(lambda x:(x.date()-reference_date.date()).days))+1
    # group by for same name & reason
    habit_action['status']=habit_action.status.apply(lambda x:int(x=='p'))
    habit_action_gpb=habit_action.groupby(['name','small_desc','habit_status','reason'],as_index=False)
    habit_action=habit_action_gpb.agg({'days_from_today':'min','status':'mean'})
    habit_action=pd.merge(habit_action,habit_rev_action,on='reason',how='left')
    habit_action['action_new']=habit_action.apply(lambda x: x.action if x.days_from_today==1 else x.action_subsequent, axis=1)
    habit_action['action']=habit_action['action_new']
    habit_action.drop(['action_new','action_subsequent'],axis=1,inplace=True)
    habit_action.sort_values(['status','days_from_today'],inplace=True)
    habit_action.action.fillna(value=list(habit_rev_action.loc[habit_rev_action.reason=='other','action'])[0],inplace=True)
    return habit_action

get_actions_against_habits(dathabitLog,habit_streak,reference_date)

,name,small_desc,habit_status,reason,days_from_today,status,action
3,goal,work towards your goal (with himmat),focus,not started,1,0,FT: check if all okay
4,gtd,complete task in timely manner,focus,not started,1,0,FT: check if all okay
5,habit,act towards making life long habit,focus,not started,1,0,FT: check if all okay
6,ilm,revise (n read) as per the plan,now,not started,1,0,FT: check if all okay
7,quran,hifz towards goal ane murajaat as per schedule,now,not started,1,0,FT: check if all okay
8,qurandaur,daily quran daur ghar ma,now,not started,1,0,FT: check if all okay
0,ft,focused work with end time,focus,forgot,3,0,swish
2,ft,focused work with end time,focus,put the mobile great job,1,1,revise
1,ft,focused work with end time,focus,marvelous work,2,1,revise


In [176]:
# Sort by: review_type, habit_status
# Color Code:
# red - focus and missed
# black - now, later and missed
# green - scheduled review
def display_habit_review(habit_review):
    

    def get_font_attribute(df):
        color = ""
        if df.review_type.lower() == 'missed':
            color = 'red' if df.habit_status == 'focus' else 'black'
        else:
            color = 'green'
        return 'color=' + color

    table_config = ({'column_name_in_order_of_display': ['name', 'small_desc', 'streak_str','habit_status','review_type'],
                     'column_headers': ['Name', 'Description', 'Streak', 'Focus', 'Review Type'],
                     'column_sort_by': ['review_type', 'habit_status'],
                     'column_sort_by_is_asc': [True, True],
                     'widths_in_pcntg': [10, 50, 20, 10, 10],
                     'text_alignment': ['left', 'left', 'center', 'center', 'center'],
                     'column_name_whose_value_change_adds_horizontal_ruler': None,
                     'column_name_needing_custom_formatting': 'small_desc',
                     'func_giving_font_attribute': get_font_attribute,
                     'table_heading': 'Habit Review'
                     })
    display_html_table(habit_review, table_config)

display_habit_review(habit_review)

Name,Description,Streak,Focus,Review Type
gussa,gussa (n ducha) ne gili jao ane action ma takheer karo,"0c (0m, -3d)",now,Missed
budget,monthly budget n expense control expense (chehlum n urs vaaz for bhalai n israfi si bacho n 10 ma 1),"0c (0m, -3d)",now,Missed
nazarulmaqam,habit of putting nazarul makam for every important thing,"0c (0m, -3d)",now,Missed
google,speed up by trying to reuse n learning from others so google,"0c (0m, -3d)",now,Missed
aitena,apni chizo ni aitena karo jem glass,"0c (0m, -3d)",now,Missed


In [165]:
# Sort by: review_type, habit_status
# Color Code:
# red - focus and missed
# black - now, later and missed
# green - scheduled review
def get_habits_to_be_reviewed(dathabit,dathabitLog):
    # Habit Reviews
    # review happens on all days of week

    common_col_names=['name','review_type']
    # No entry in log - Review
    habit_last_entered_date=dathabitLog.groupby('name',as_index=False).agg({'date':'max'})
    habit_last_entered_date['days_from_last_entry']=habit_last_entered_date.date.apply(lambda x,reference_date: (reference_date-x).days,args=[reference_date])
    habit_last_entered_date.drop(['date'],inplace=True,axis=1)
    habit_missed_review=pd.merge(dathabit[['id','name','streak_type','missed_review_cycle']],habit_last_entered_date,on='name',how='left')
    habit_missed_review=habit_missed_review.loc[habit_missed_review.streak_type=='count',:]
    habit_missed_review.days_from_last_entry.fillna(999,inplace=True) # if no entry found, add a big number
    habit_missed_review['days_from_last_entry']=habit_missed_review.days_from_last_entry.astype('int',copy=True)
    habit_missed_review=habit_missed_review.loc[habit_missed_review.days_from_last_entry>=habit_missed_review.missed_review_cycle]
    habit_missed_review['review_type']='Missed'
    habit_missed_review=habit_missed_review.loc[:,common_col_names]
    habit_missed_review

    # Scheduled Reviews
    def is_review_date_reached(habit_id,reference_date,review_cycle):
        offset_to_balance_load_across_days=habit_id%review_cycle # add 1 to avoid 0 date
        number_of_days_from_year_start=(reference_date.date()-dt.datetime.strptime(str(reference_date.year)+"/01/01","%Y/%m/%d").date()).days
        eligible_review_days=[offset_to_balance_load_across_days+x*review_cycle for x in range(int(365/review_cycle))]
        return (number_of_days_from_year_start in eligible_review_days)

    habit_review=dathabit[dathabit.apply(lambda x,reference_date: is_review_date_reached(x.id,reference_date,x.review_cycle),args=[reference_date],axis=1)]
    habit_review['review_type']='Scheduled'
    habit_review=habit_review.loc[:,common_col_names]
    habit_review=habit_review.loc[habit_review.name.isin(habit_missed_review.name)==False]

    habit_review=pd.concat([habit_review,habit_missed_review])
    habit_review=pd.merge(habit_review,habit_status,on='name',how='inner')
    habit_review.drop(['current_streak','current_streak_by_total_days'],inplace=True,axis=1)
    return habit_review

get_habits_to_be_reviewed(dathabit,dathabitLog)

,name,review_type,small_desc,habit_status,streak_str
0,gussa,Missed,gussa (n ducha) ne gili jao ane action ma takh...,now,"0c (0m, -3d)"
1,budget,Missed,monthly budget n expense control expense (cheh...,now,"0c (0m, -3d)"
2,nazarulmaqam,Missed,habit of putting nazarul makam for every impor...,now,"0c (0m, -3d)"
3,google,Missed,speed up by trying to reuse n learning from ot...,now,"0c (0m, -3d)"
4,aitena,Missed,apni chizo ni aitena karo jem glass,now,"0c (0m, -3d)"


ModuleNotFoundError: No module named 'jupyterNbs'